In [1]:
import random
import pygame
import sys

# Constants
TILE_SIZE = 24
GRID_WIDTH = 30
GRID_HEIGHT = 20
SCREEN_WIDTH = TILE_SIZE * GRID_WIDTH
SCREEN_HEIGHT = TILE_SIZE * GRID_HEIGHT
FPS = 3  # Increase for faster movement

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Fireboy and Watergirl Grid World")
clock = pygame.time.Clock()

# Colors
COLORS = {
    "0": (120, 66, 18),   # Brown - Empty
    "1": (160, 82, 45),   # Light Brown - Platform
    "F": (255, 0, 0),     # Fireboy - Red Heart
    "W": (0, 0, 255),     # Watergirl - Blue Heart
    "G": (0, 255, 0),     # Green Agent
    "RP": (255, 0, 0),    # Red Poison
    "BP": (0, 0, 255),    # Blue Poison
    "GP": (0, 255, 0),    # Green Poison
    "FG": (255, 0, 0),    # Fireboy Goal
    "WG": (0, 0, 255),    # Watergirl Goal
    "RB": (255, 0, 0),    # Red Gem
    "BB": (0, 0, 255),    # Blue Gem
    "YS": (255, 255, 0),  # Yellow Slide
    "PS": (128, 0, 128),  # Purple Slide
    "YB": (255, 255, 0),  # Yellow Button
    "PB": (128, 0, 128),  # Purple Button
}

BASE_COLOR = COLORS["0"]

# Sample grid
grid = [["0" for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)]

# Place agents
agents = {
    "F": [18, 1],
    "W": [15, 1],
    "G": [2, 19]
}

for agent, pos in agents.items():
    grid[pos[0]][pos[1]] = agent

# Poison tiles
for x in range(10, 13): grid[18][x] = "RP"
for x in range(17, 20): grid[18][x] = "BP"
for x in range(18, 21): grid[14][x] = "GP"

# Gems
grid[17][11] = grid[8][10] = "RB"
grid[17][18] = grid[10][20] = "BB"

# Goals
grid[2][26] = "FG"
grid[2][28] = "WG"

# Slides
for x in range(1, 4): grid[9][x] = "YS"
for x in range(26, 29): grid[6][x] = "PS"

# Buttons
grid[8][6] = grid[12][9] = "YB"
grid[8][14] = grid[5][22] = "PB"

# Platforms (walls)
for i in range(30): grid[0][i] = grid[19][i] = "1"
for i in range(20): grid[i][0] = grid[i][29] = "1"
grid[1][1] = grid[1][2] = "1"
for i in range(4, 29): grid[3][i] = "1"
grid[4][1] = grid[4][2] = grid[5][1] = grid[5][2] = "1"
for i in range(1, 26): grid[6][i] = "1"
grid[7][18] = grid[7][19] = grid[7][20] = "1"
grid[8][18] = grid[8][19] = grid[8][20] = "1"
for i in range(4, 16): grid[9][i] = "1"
for i in range(17, 22): grid[11][i] = "1"
for i in range(22, 29): grid[10][i] = "1"
grid[10][16] = "1"
grid[11][26] = grid[11][27] = grid[11][28] = "1"
grid[12][26] = grid[12][27] = grid[12][28] = "1"
for i in range(1, 13): grid[13][i] = "1"
grid[14][13] = "1"
for i in range(14, 27): grid[15][i] = "1"
for i in range(1, 7): grid[16][i] = "1"
grid[17][27] = grid[17][28] = grid[18][27] = grid[18][28] = "1"

# Drawing function
def draw_grid():
    for y in range(GRID_HEIGHT):
        for x in range(GRID_WIDTH):
            tile = grid[y][x]
            color = COLORS.get(tile, BASE_COLOR)

            # Agent hearts
            if tile in ["F", "W", "G"]:
                pygame.draw.rect(screen, color, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
                pygame.draw.polygon(screen, color, [
                    (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE + 6),
                    (x*TILE_SIZE + 6, y*TILE_SIZE + TILE_SIZE//2),
                    (x*TILE_SIZE + TILE_SIZE - 6, y*TILE_SIZE + TILE_SIZE//2),
                ])
                pygame.draw.circle(screen, color, (x*TILE_SIZE + TILE_SIZE//3, y*TILE_SIZE + TILE_SIZE//3), 5)
                pygame.draw.circle(screen, color, (x*TILE_SIZE + 2*TILE_SIZE//3, y*TILE_SIZE + TILE_SIZE//3), 5)

            # Gems
            elif tile in ["RB", "BB"]:
                pygame.draw.rect(screen, BASE_COLOR, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
                pygame.draw.polygon(screen, color, [
                    (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE),
                    (x*TILE_SIZE + TILE_SIZE, y*TILE_SIZE + TILE_SIZE//2),
                    (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE + TILE_SIZE),
                    (x*TILE_SIZE, y*TILE_SIZE + TILE_SIZE//2)
                ])

            # Buttons
            elif tile in ["YB", "PB"]:
                pygame.draw.rect(screen, BASE_COLOR, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
                pygame.draw.circle(screen, color, (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE + TILE_SIZE//2), TILE_SIZE//4)

            # Goals
            elif tile in ["FG", "WG"]:
                pygame.draw.rect(screen, BASE_COLOR, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))
                pygame.draw.polygon(screen, color, [
                    (x*TILE_SIZE + TILE_SIZE//2, y*TILE_SIZE),
                    (x*TILE_SIZE, y*TILE_SIZE + TILE_SIZE),
                    (x*TILE_SIZE + TILE_SIZE, y*TILE_SIZE + TILE_SIZE)
                ])

            else:
                pygame.draw.rect(screen, color, (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE))

            pygame.draw.rect(screen, (160, 82, 45), (x*TILE_SIZE, y*TILE_SIZE, TILE_SIZE, TILE_SIZE), 1)

# Movement check
def is_valid_move(y, x):
    return (
        0 <= x < GRID_WIDTH and
        0 <= y < GRID_HEIGHT and
        grid[y][x] == "0"
    )

# Random movement
def move_agents_randomly():
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    for agent, pos in agents.items():
        y, x = pos
        random.shuffle(directions)
        for dy, dx in directions:
            ny, nx = y + dy, x + dx
            if is_valid_move(ny, nx):
                grid[y][x] = "0"
                grid[ny][nx] = agent
                agents[agent] = [ny, nx]
                break

# Game loop
running = True
while running:
    screen.fill((0, 0, 0))
    move_agents_randomly()
    draw_grid()
    pygame.display.flip()
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\Srinu\anaconda3\envs\myenv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
